# Classification with Decision Trees

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from sklearn.inspection import DecisionBoundaryDisplay

from sklearn.linear_model import LogisticRegression

import sklearn.tree
from sklearn.tree import DecisionTreeClassifier

import dtreeviz

Using make_blobs to generate some data for classification:
https://docs.w3cub.com/scikit_learn/modules/generated/sklearn.datasets.make_blobs

In [ ]:
x, y = make_blobs(n_samples=300,
                  random_state=0, 
                  cluster_std=2)

In [ ]:
plt.scatter(x[:,0], x[:,1], c=y);

## Train/test split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, 
                                                    y, 
                                                    random_state=0)

## First, Logistic Regression

In [ ]:
logreg_clf = LogisticRegression()
logreg_clf.fit(x_train, y_train)

In [ ]:
DecisionBoundaryDisplay.from_estimator(logreg_clf, 
                                       x,
                                       response_method="predict",
                                       cmap="RdBu", 
                                       alpha=0.5
)

plt.scatter(x[:,0], x[:,1], c=y);

In [ ]:
test_score = logreg_clf.score(x_test, y_test)
print(f"Accuracy of Logistic Regression: {test_score:.2f}")

## Decision Tree

In [ ]:
tree_clf = DecisionTreeClassifier()

# tree_clf = DecisionTreeClassifier(max_depth=2)

In [ ]:
tree_clf.fit(x_train, y_train)

In [ ]:
DecisionBoundaryDisplay.from_estimator(tree_clf, 
                                       x,
                                       response_method="predict",
                                       cmap="RdBu", 
                                       alpha=0.5
)

plt.scatter(x[:,0], x[:,1], c=y);

In [ ]:
text_representation = sklearn.tree.export_text(tree_clf)
print(text_representation)

In [ ]:
tree_clf.classes_

In [ ]:
plt.figure(figsize=(12,8))
sklearn.tree.plot_tree(tree_clf, 
               feature_names=['x0','x1'],  
               class_names=[str(i) for i in tree_clf.classes_],
               filled=True);

In [ ]:
%%capture --no-display

vizmodel = dtreeviz.model(tree_clf, 
         x,
         y,
         feature_names=['x0','x1'],
         class_names=[i for i in tree_clf.classes_],
         target_name="y")

vizmodel.view()

In [ ]:
sample = [0., 1.]
print(vizmodel.explain_prediction_path(sample))

In [ ]:
test_score = tree_clf.score(x_test, y_test)
print(f"Accuracy of Decision Tree: {test_score:.2f}")

### How to make predictions?

In [ ]:
sample = [[0,1]]
print('Predicted Class:',tree_clf.predict(sample))

In [ ]:
y_pred_proba = tree_clf.predict_proba(sample)
print(y_pred_proba)

In [ ]:
plt.bar(tree_clf.classes_, y_pred_proba[0])

# Breast Cancer Dataset

Let's adapt the above to do Logistic Regression and Decision Tree Classification on the breast cancer dataset.

* Use the feature variables "mean radius" and "mean texture"
* Predict the 0 or 1 cancer target variable

In [ ]:
import sklearn.datasets
import sklearn.model_selection
import pandas as pd
import seaborn as sns

In [ ]:
x,y = sklearn.datasets.load_breast_cancer(return_X_y=True,
                                          as_frame=True)

In [ ]:
x['target'] = y

In [ ]:
x.corr()['target'].sort_values().plot.barh(xlim=[-1.0,1.0])

In [ ]:
sns.scatterplot(data=x, 
                x='mean radius', 
                y='mean texture',
                hue='target')

In [ ]:
# split into training/test sets

x_train, x_test, y_train, y_test = train_test_split(x[['mean radius','mean texture']], 
                                                    x['target'], 
                                                    random_state=0)

In [ ]:
# initialize the logistic regression model

logreg_clf = LogisticRegression()

In [ ]:
# train the logistic regression model

logreg_clf.fit(x_train, y_train)

In [ ]:
# plot the decision boundary with the data

DecisionBoundaryDisplay.from_estimator(logreg_clf, 
                                       x[['mean radius','mean texture']], 
                                       response_method="predict",
                                       cmap="RdBu", 
                                       alpha=0.5
)

sns.scatterplot(data=x,
                x='mean radius', 
                y='mean texture',
                hue='target',
                palette=['red','green'])

plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

In [ ]:
# print the accuracy of the model

test_score = logreg_clf.score(x_test, y_test)
print(f"Accuracy of Logistic Regression: {test_score:.2f}")

In [ ]:
# initialize the decision tree classification model

tree_clf = DecisionTreeClassifier()

In [ ]:
# train the decision tree classification model

tree_clf.fit(x_train, y_train)

In [ ]:
# plot the decision boundary with the data

DecisionBoundaryDisplay.from_estimator(tree_clf, 
                                       x[['mean radius','mean texture']], 
                                       response_method="predict",
                                       cmap="RdBu", 
                                       alpha=0.5
)

sns.scatterplot(data=x,
                x='mean radius', 
                y='mean texture',
                hue='target',
                palette=['red','green'])

plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

In [ ]:
# print out a text representation of the tree

text_representation = sklearn.tree.export_text(tree_clf)
print(text_representation)

In [ ]:
# plot a visualization of the tree

tree_clf.classes_

In [ ]:
# print the accuracy of the model

plt.figure(figsize=(12,8))
sklearn.tree.plot_tree(tree_clf, 
               feature_names=['mean radius','mean texture'],  
               class_names=[str(i) for i in tree_clf.classes_],
               filled=True);

In [ ]:
# make a sample data point
# predict its class
# and print the calculated prediction probabilities for each class

sample = pd.DataFrame({"mean radius": [16.], "mean texture": [40]})
print('Predicted Class:',tree_clf.predict(sample))

y_pred_proba = tree_clf.predict_proba(sample)
print(y_pred_proba)

y_proba_class_0 = pd.Series(y_pred_proba[0], index=tree_clf.classes_).plot.bar()

test_score = tree_clf.score(x_test, y_test)
print(f"Accuracy of Decision Tree: {test_score:.2f}")

In [ ]:
import ipywidgets

In [ ]:
def dt(m=3):

    tree_clf = DecisionTreeClassifier(max_depth=m)
    tree_clf.fit(x_train, y_train)
    
    DecisionBoundaryDisplay.from_estimator(tree_clf, 
                                           x[['mean radius','mean texture']], 
                                           response_method="predict",
                                           cmap="RdBu", 
                                           alpha=0.5
    )

    sns.scatterplot(data=x,
                    x='mean radius', 
                    y='mean texture',
                    hue='target',
                    palette=['red','green'])

    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    test_score = tree_clf.score(x_test, y_test)
    print(f"Accuracy of Decision Tree: {test_score:.2f}")
    
ipywidgets.interact(dt,m=(1,10));